# Recurrent Neural Networks

Humans don’t start their thinking from scratch every second. As you read this essay, you understand each word based on your understanding of previous words. You don’t throw everything away and start thinking from scratch again. Your thoughts have persistence.

Traditional neural networks can’t do this, and it seems like a major shortcoming. For example, imagine you want to classify what kind of event is happening at every point in a movie. It’s unclear how a traditional neural network could use its reasoning about previous events in the film to inform later ones.

Recurrent neural networks address this issue. They are networks with loops in them, allowing information to persist.

<img src='./images/rnn_rolled.png' width="150">

#### Recurrent Neural Networks have loops.

[image source] https://colah.github.io/posts/2015-08-Understanding-LSTMs/

In the above diagram, a chunk of neural network, A, looks at some input xt and outputs a value ht

A loop allows information to be passed from one step of the network to the next.

These loops make recurrent neural networks seem kind of mysterious. However, if you think a bit more, it turns out that they aren’t all that different than a normal neural network. A recurrent neural network can be thought of as multiple copies of the same network, each passing a message to a successor. Consider what happens if we unroll the loop:

<img src='./images/rnn_unrolled.png' width="700">

#### An unrolled recurrent neural network.

[image source] https://colah.github.io/posts/2015-08-Understanding-LSTMs/

This chain-like nature reveals that recurrent neural networks are intimately related to **sequences and lists**. They’re the natural architecture of neural network to use for such data.

And they certainly are used! In the last few years, there have been incredible success applying RNNs to a variety of problems: **speech recognition, language modeling, translation, image captioning...** The list goes on. 

Essential to these successes is the use of **“LSTMs,”** a very special kind of recurrent neural network which works, for many tasks, much much better than the standard version. Almost all exciting results based on recurrent neural networks are achieved with them. It’s these LSTMs that this essay will explore.

## The Problem of Long-Term Dependencies

One of the appeals of RNNs is the idea that they might be able to connect previous information to the present task, such as using previous video frames might inform the understanding of the present frame. If RNNs could do this, they’d be extremely useful. But can they? It depends.

**Sometimes, we only need to look at recent information to perform the present task**. For example, consider a language model trying to predict the next word based on the previous ones. If we are trying to predict the last word in “the clouds are in the sky,” we don’t need any further context – it’s pretty obvious the next word is going to be sky. In such cases, where the gap between the relevant information and the place that it’s needed is small, RNNs can learn to use the past information.

<img src='./images/rnn_shorttermdepdencies.png' width="500">

#### An unrolled recurrent neural network.

[image source] https://colah.github.io/posts/2015-08-Understanding-LSTMs/

**But there are also cases where we need more context.** Consider trying to predict the last word in the text “I grew up in France… I speak fluent French.” Recent information suggests that the next word is probably the name of a language, but if we want to narrow down which language, we need the context of France, from further back. It’s entirely possible for the gap between the relevant information and the point where it is needed to become very large.

**Unfortunately, as that gap grows, RNNs become unable to learn to connect the information.**

<img src='./images/rnn_longtermdependencies.png' width="600">

#### An unrolled recurrent neural network.

[image source] https://colah.github.io/posts/2015-08-Understanding-LSTMs/

In theory, RNNs are absolutely capable of handling such “long-term dependencies.” A human could carefully pick parameters for them to solve toy problems of this form. Sadly, in practice, RNNs don’t seem to be able to learn them. The problem was explored in depth by [Hochreiter (1991) German](http://people.idsia.ch/~juergen/SeppHochreiter1991ThesisAdvisorSchmidhuber.pdf) and [Bengio, et al. (1994)](http://ai.dinfo.unifi.it/paolo//ps/tnn-94-gradient.pdf), who found some pretty fundamental reasons why it might be difficult.

Thankfully, LSTMs don’t have this problem!

## Long Short Term Memory (LSTM) Networks

Long Short Term Memory networks – usually just called “LSTMs” – are a special kind of RNN, capable of learning long-term dependencies. They were introduced by [Hochreiter & Schmidhuber (1997)](http://www.bioinf.jku.at/publications/older/2604.pdf), and were refined and popularized by many people in following work.1 They work tremendously well on a large variety of problems, and are now widely used.

LSTMs are explicitly designed to avoid the **long-term dependency problem**. Remembering information for long periods of time is practically their default behavior, not something they struggle to learn!

All recurrent neural networks have the form of a chain of repeating modules of neural network. In standard RNNs, this repeating module will have a very simple structure, such as a single tanh layer.

### Standard RNNs

<img src='./images/lstm_simplernn.png' width="600">

#### The repeating module in a standard RNN contains a single layer.

[image source] https://colah.github.io/posts/2015-08-Understanding-LSTMs/

### LSTMs

<img src='./images/lstm_chain.png' width="600">

#### The repeating module in an LSTM contains four interacting layers.

[image source] https://colah.github.io/posts/2015-08-Understanding-LSTMs/

Don’t worry about the details of what’s going on. We’ll walk through the LSTM diagram step by step later. For now, let’s just try to get comfortable with the notation we’ll be using.

<img src='./images/lstm_notation.png' width="600">

#### The repeating module in an LSTM contains four interacting layers.

[image source] https://colah.github.io/posts/2015-08-Understanding-LSTMs/

In the above diagram, each line carries an entire vector, from the output of one node to the inputs of others. The pink circles represent pointwise operations, like vector addition, while the yellow boxes are learned neural network layers. Lines merging denote concatenation, while a line forking denote its content being copied and the copies going to different locations. 

## The Core Idea Behind LSTMs

**The key to LSTMs is the cell state, the horizontal line running through the top of the diagram.**

The cell state is kind of like a conveyor belt. It runs straight down the entire chain, with only some minor linear interactions. It’s very easy for information to just flow along it unchanged.

<img src='./images/lstm_c_line.png' width="800">


[image source] https://colah.github.io/posts/2015-08-Understanding-LSTMs/

Gates are a way to optionally let information through. They are composed out of a sigmoid neural net layer and a pointwise multiplication operation.

<img src='./images/lstm_gate.png' width="200">


[image source] https://colah.github.io/posts/2015-08-Understanding-LSTMs/

** The sigmoid layer outputs numbers between zero and one, describing how much of each component should be let through **. A value of zero means “let nothing through,” while a value of one means “let everything through!”

An LSTM has three of these gates, to protect and control the cell state.

## Step-by-Step LSTM Walk Through

**The first step in our LSTM is to decide what information we’re going to throw away from the cell state**. This decision is made by a sigmoid layer called the “forget gate layer.” It looks at ht−1 and xt, and outputs a number between 0 and 1 for each number in the cell state Ct−1. A 1 represents “completely keep this” while a 0 represents “completely get rid of this.”

Let’s go back to our example of a language model trying to predict the next word based on all the previous ones. In such a problem, the cell state might include the gender of the present subject, so that the correct pronouns can be used. When we see a new subject, we want to forget the gender of the old subject.

<img src='./images/lstm_f_gate.png' width="800">


[image source] https://colah.github.io/posts/2015-08-Understanding-LSTMs/

**The next step is to decide what new information we’re going to store in the cell state**. This has two parts. First, a sigmoid layer called the “input gate layer” decides which values we’ll update. Next, a tanh layer creates a vector of new candidate values, C̃ t, that could be added to the state. In the next step, we’ll combine these two to create an update to the state.

In the example of our language model, we’d want to add the gender of the new subject to the cell state, to replace the old one we’re forgetting.

<img src='./images/lstm_i_gate.png' width="800">


[image source] https://colah.github.io/posts/2015-08-Understanding-LSTMs/

**It’s now time to update the old cell state, Ct−1, into the new cell state Ct**. The previous steps already decided what to do, we just need to actually do it. We multiply the old state by ft, forgetting the things we decided to forget earlier. Then we add it∗C̃ t. This is the new candidate values, scaled by how much we decided to update each state value.

In the case of the language model, this is where we’d actually drop the information about the old subject’s gender and add the new information, as we decided in the previous steps.

<img src='./images/lstm_c_gate.png' width="800">


[image source] https://colah.github.io/posts/2015-08-Understanding-LSTMs/

**Finally, we need to decide what we’re going to output**. This output will be based on our cell state, but will be a filtered version. First, we run a sigmoid layer which decides what parts of the cell state we’re going to output. Then, we put the cell state through tanh (to push the values to be between −1 and 1) and multiply it by the output of the sigmoid gate, so that we only output the parts we decided to.

For the language model example, since it just saw a subject, it might want to output information relevant to a verb, in case that’s what is coming next. For example, it might output whether the subject is singular or plural, so that we know what form a verb should be conjugated into if that’s what follows next.

<img src='./images/lstm_o_gate.png' width="800">


[image source] https://colah.github.io/posts/2015-08-Understanding-LSTMs/